# HLCV Assignment 2

### Prequel
This notebook contains skeletons that you are supposed to fill in (watch out for <font color='red'>**TODO**</font> tags). In general, you are expected to implement methods yourself. For example, when asked to implement a method computing histograms you can use numpy array functions that help you implement this task, but not use the numpy implementation `np.histogram` for your solution. To submit your assignment, upload your completed notebook and a PDF report (use the provided Latex template with, for example, [Overleaf](http://overleaf.com)) that contains your observations.

### Setting
In this exercise you will train (i) shallow classifiers and (ii) deep learning-based classifiers on a dataset containing 10 different types of clothing articles and compare their performance with each other.

In [ ]:
import torch, torchvision, matplotlib.pyplot as plt, sklearn, numpy as np, tqdm.notebook as tqdm, time
from IPython. display import clear_output

# Part I: SVM and Decision Boundaries

In this part of the exercise you will get better acquainted with the SVM algorithm using an artificially generated 2D classification problem. As you might recall from the lecture the problem setting is the following. We are given a training dataset $\{(X_i, y_i), i = 1, . . . ,N\}$, where $X_i$ is an *M*-dimensional feature vector and $y_i \in \{−1, 1\}$ is a label which denotes whether the corresponding point should be classified as negative or as positive. Further, we assume that the classification boundary has the form $y(X) = W^TX + w_0$.

The new point is classified as positive if $y(X) > 0$ and negative otherwise.

You can use the provided function to generate a simple 2D dataset in which positive and negative training points are sampled from 2 distinct Gaussian distributions. The last two parameters of this function allow to control the variance of the points, so that both linearly separable and linearly non-separable datasets can be generated.

### a) SVM Training

Use the following function to generate 2D Gaussian data and plot it.

In [ ]:
def generate_gauss_data(N, mean_pos, mean_neg, std_pos, std_neg):
    state = np.random.RandomState(seed=42)
    pos = np.stack((
        state.normal(size=N, loc=mean_pos, scale=std_pos),
        state.normal(size=N, loc=0, scale=1)
    )).T
    neg = np.stack((
        state.normal(size=N, loc=mean_neg, scale=std_neg),
        state.normal(size=N, loc=0, scale=1)
    )).T
    return pos, neg

In [ ]:
data_pos, data_neg = generate_gauss_data(
    N = 500,
    mean_pos = -4,
    mean_neg =  4,
    std_pos = 1,
    std_neg = 1
)
plt.scatter(data_pos[:,0], data_pos[:,1])
plt.scatter(data_neg[:,0], data_neg[:,1])
plt.xlabel("x1")
plt.ylabel("x2")
plt.grid(linestyle="--")
plt.show()

<font color='red'>**TODO:**</font> The following function trains an SVM on your generated Gaussian data. Make yourself familiar with the parameters of sklearn's SVM implementation (see https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html). Then, run the code.

In [ ]:
def train_svm(data_pos, data_neg, svm_pars):
    data = np.concatenate((data_pos, data_neg))
    labels = np.concatenate((
        np.ones(len(data_pos)),
        -np.ones(len(data_neg))
    ))

    clf = sklearn.svm.SVC(**svm_pars)
    return clf.fit(X=data, y=labels)

In [ ]:
clf = train_svm(
    data_pos = data_pos,
    data_neg = data_neg,
    svm_pars  = {
        "kernel": "linear",
        "probability": True
    }
)

### b) SVM Visualization

<font color='red'>**TODO:**</font> Adjust the next cell to not only plot the training dataset, but also the support vectors and decision boundary of the trained SVM. The attribute section of the SVM documentary might help you with this.

In [ ]:
def plot_svm(data_pos, data_neg, clf, title=None):
    # plot the training data
    
    plt.scatter(data_pos[:,0], data_pos[:,1])
    plt.scatter(data_neg[:,0], data_neg[:,1])

    # plot the decision boundary

    # ...

    # plot the support vectors

    # ...
    
    plt.grid(linestyle="--")
    plt.title(title)
    plt.show()

In [ ]:
plot_svm(data_pos=data_pos, data_neg=data_neg, clf=clf)

### c) Linearly non-separable data
<font color='red'>**TODO:**</font> Find a setting where the data is not linearly separable anymore by changing the standard deviations.

In [ ]:
data_pos, data_neg = generate_gauss_data(
    # add your parameters here
)
plt.scatter(data_pos[:,0], data_pos[:,1])
plt.scatter(data_neg[:,0], data_neg[:,1])
plt.xlabel("x1")
plt.ylabel("x2")
plt.grid(linestyle="--")
plt.show()

<font color='red'>**TODO:**</font> Train different SVM on the linearly non-separable data by varying their *C* parameter. Visualize their resulting decision boundaries and support vectors. What do you observe? What is the role of *C* in the SVM classification algorithm?

In [ ]:
# add your code here

# Part II: Image Classification with Fashion-MNIST

### Download Fashion-MNIST and show samples.
[Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) is a (more challenging MNIST-like) dataset that contains 28x28 grayscale images of 10 different types of clothing articles, hence **10** classes. There is a training dataset containing 60k images and a test dataset containing 10k images. Torchvision provides functionality to download both datasets (see next cell), so it is sufficient to run the code.

In [ ]:
dataset_train = torchvision.datasets.FashionMNIST("data", train=True,  download=True)
dataset_test  = torchvision.datasets.FashionMNIST("data", train=False, download=True)

Lets extract the images into numpy arrays. Also, show some sample images.

In [ ]:
X_train = dataset_train.data.numpy()
y_train = dataset_train.targets.numpy()
X_test  = dataset_test.data.numpy()
y_test  = dataset_test.targets.numpy()

print(X_train.shape, X_test.shape)

f, ax = plt.subplots(10,10, figsize=(15,15))
for c in range(10):
    for i in range(10):
        ax[c,i].imshow(X_train[y_train==c][i], interpolation="none", cmap="gray")
        if i==0:
            ax[c,i].set_ylabel(dataset_train.classes[c])
        if i==9:
            ax[c,i].yaxis.set_label_position("right")
            ax[c,i].set_ylabel(f"c={c}")
        ax[c,i].set_yticks([])
        ax[c,i].set_xticks([])
plt.show()

We split the training dataset into a split used for training (50k images) and another split used for validation (10k images).

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train,
    test_size = 10_000,
    random_state = 42,
    stratify = y_train
)

np.unique(y_val, return_counts=True)

### a) Shallow Learning - Image as input features
<font color='red'>**TODO:**</font> Choose at least 3 classifiers (of which one is Logistic Regression) from sklearn and train these models on the **training split**. Tune possible hyperparameters on the **validation split**. Finally, report your best found model (according to validation) with their corresponding performance (i) on the **training split**, (ii) on the **validation split**, and (iii) on the **test data**. Use `random_state=42` when initializing non-deterministic classifiers to get reproducable results. Compose your results (and found hyperparameters) in a table in your report.

You can find possible classifiers here: https://scikit-learn.org/stable/supervised_learning.html

Train, tune, and test different classifiers below.

In [ ]:
# Here is an example printing the number of correctly classified training samples of a Logistic Regression.

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=42, max_iter=200)
clf.fit(
    X = X_train.reshape(len(X_train), -1),
    y = y_train
)
pred_train = clf.predict(X=X_train.reshape(len(X_train), -1))
print((pred_train==y_train).sum())

In [ ]:
# Train and tune classifiers

In [ ]:
# Approximate the generalization error of your best found hyperparameter combinations for each classifier

### b) Deep Learning - Learn to Extract Features from Images
### Defining our classification network.

In [ ]:
class CNNClassifier(torch.nn.Module):
    def __init__(self, kernel_size=3, out_channels=16, pool_size=7):
        # inherit from torch.nn.Module
        super(type(self), self).__init__()
        assert (28%pool_size) == 0, "pool_size must be a divisible of image dimensions (28)"
        self.conv = torch.nn.Conv2d(
            kernel_size = kernel_size,
            in_channels = 1,
            out_channels = out_channels,
            padding = "same"
        )
        self.linear = torch.nn.Linear(out_channels*((28//pool_size)**2), 10)
        self.pool_size = pool_size
        
    def forward(self, x):
        # input image size: x.shape = Bx1x28x28 (Batchsize x Channels x Height x Width)
        x = self.conv(x)
        x = torch.nn.functional.relu(x)
        # feature maps size: x.shape = Bx16x28x28
        # reduce spatial dimensionality of feature maps
        x = torch.nn.functional.avg_pool2d(x, kernel_size=self.pool_size)
        # feature maps size: x.shape = Bx16x4x4
        # flatten features for input to linear layer
        x_feat = x.reshape(len(x), -1)
        # dimensionality of features is now 16*4*4 = Bx256
        # previously, the dimensionality of inputting the plain images was: 28*28=784
        y_logits = self.linear(x_feat)
        # now we have prediction scores for each class: x.shape == Bx10
        return y_logits, x_feat

    def reset_parameters(self):
        self.conv.reset_parameters()
        self.linear.reset_parameters()

### Training our classification network.

The following cells contain utility functions.
- `plot_perf`: You can collect your measures into lists and use the following function to plot your results.
- `save_state`: Use this function to save the state of a network.
- `load_state`: Use this function to create a CNNClassifier from a saved state.

In [ ]:
def plot_perf(train:list, val:list , name:list, share_axis=False):
    assert len(train) == len(val) == len(name)
    
    f, host = plt.subplots(figsize=(8,5))
    handles = []
    colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
    
    plt.xlabel("Epoch")
    plt.grid(linestyle="--")
    
    for i in range(len(train)):
        ax = plt.gca()
        if not share_axis:
            if i == 0:
                ax = host
            else:
                ax = host.twinx()
            ax.set_ylabel(name[i], c=colors[i])
            ax.tick_params(axis="y", colors=colors[i])
        
        handles += ax.plot(train[i], label=f"{name[i]}, train", c=colors[i])
        handles += ax.plot(val[i], label=f"{name[i]}, val", c=colors[i], linestyle="--")
    
    plt.legend(handles=handles, bbox_to_anchor=(1.04, 1), loc="upper left")
    plt.show()

In [ ]:
def save_state(network, optimizer, name):
    torch.save({
        "net": network.state_dict(),
        "optimizer": optimizer.state_dict(),
        "params": {
            "kernel_size": network.conv.kernel_size,
            "out_channels": network.conv.out_channels,
            "pool_size": network.pool_size
        },
        "measures": {measures}
    }, "savestate_{name}.torch")

In [ ]:
def load_state(name, map_location):
    load = torch.load("savestate_{name}.torch", map_location=map_location)
    network = CNNClassifier(**load["params"])
    network.load_state(load["net"])
    return network, load["optimizer"], measures

Preparing data for Pytorch.

In [ ]:
use_gpu = True

In [ ]:
data_train, data_val = torch.from_numpy(X_train).float()/255, torch.from_numpy(X_val).float()/255
# Pytorch models expect input in the shape of BxCxHxW, therefore we add a channel dimension
data_train, data_val = data_train.unsqueeze(dim=1), data_val.unsqueeze(dim=1)
data_test, target_test = (torch.from_numpy(X_test).float()/255).unsqueeze(1), torch.from_numpy(y_test)
target_train, target_val = torch.from_numpy(y_train), torch.from_numpy(y_val)

dataset_train = torch.utils.data.TensorDataset(data_train, target_train)
dataset_val   = torch.utils.data.TensorDataset(data_val,   target_val)
dataset_test  = torch.utils.data.TensorDataset(data_test,  target_test)

dataloader_train = torch.utils.data.DataLoader(
    dataset = dataset_train,
    batch_size = 64,
    shuffle = True,
)

dataloader_val = torch.utils.data.DataLoader(
    dataset = dataset_val,
    batch_size = 64,
    shuffle = False,
)

dataloader_test = torch.utils.data.DataLoader(
    dataset = dataset_test,
    batch_size = 64,
    shuffle = False,
)

In [ ]:
models = {}

Create the network.

In [ ]:
network = CNNClassifier(
    kernel_size = 3,
    out_channels = 100,
    pool_size = 7
)
# move to GPU if GPU is available
if use_gpu and torch.cuda.is_available():
    network = network.cuda()

print(network)
print()
print("\n".join([f"{name}: {p.shape} -> {p.numel()} parameters" for name, p in network.named_parameters()]))
print()
print(f"Total number of parameters: {sum([p.shape.numel() for p in network.parameters()])}")

Create optimizer and loss function.

In [ ]:
optimizer = torch.optim.Adam(network.parameters())
loss = torch.nn.CrossEntropyLoss()

### Training loop

<font color='red'>**TODO:**</font> Adjust the training loop by implementing the following functionalities:
- Currently, the only measurements collected is the training loss. Add other performance measures.
- Currently, the validation data is not evaluated. Add performance measures for the validation data.

You can use function `plot_perf` to plot your measurements and `save_state` to save your network at any time (e.g. when the validation measures are good).

<font color='red'>**TODO:**</font> Find a combination of hyperparameters that works well for your network. Report the hyperparameters you tried with their corresponding training and validation measures in a table in your report.

In [ ]:
network.reset_parameters()

train_losses = []
val_losses = []
# example to show how plot_perf works, delete the following two lines after you understand the function
train_measurement_x = [1,2,3,4,5]
val_measurement_x = [2,1,4,3,5]
time_start = time.time()
epochs = 100

for epoch in tqdm.trange(epochs):
    train_loss = 0
    
    # 1. Train the network
    for x, y in dataloader_train:
        # delete previous gradient calculations
        optimizer.zero_grad()
        # move to GPU if GPU is available
        if use_gpu and torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()
        
        # predict class logits for each image in the batch
        y_pred, x_feat = network(x)
        # compute the loss
        L = loss(y_pred, y)
        # compute gradients
        L.backward()
        # change parameters of the network
        optimizer.step()
        
        train_loss += L.item()
        
    train_loss /= len(dataloader_train.dataset)
    
    # 2. Validate the network
    with torch.no_grad(): # we don't need gradients: speeds up computations
        for x, y in dataloader_val:
            ...
    
    # 3. collect all measures
    train_losses += [train_loss]
    
    # clear cell output to replot progress
    clear_output(wait=True)
    
    # some output
    print(f"[{epoch:>2}] Train loss: {train_loss:.5f}")
    time_past = time.time() - time_start
    time_per_epoch = time_past / (epoch+1)
    eta = (epochs-1-epoch) * time_per_epoch
    print(f"[{epoch:>2}] Time: {time_past:.1f}s, Time-per-Epoch: {time_per_epoch:.1f}s, eta: {eta:.1f}s")
    
    # takes lists of lists as input:
    # each item of argument ´train´ expects a corresponding item in ´val´.
    plot_perf(
        train = [train_losses, train_measurement_x],
        val = [val_losses, val_measurement_x],
        name = ["loss", "measurement x"]
    )

<font color='red'>**TODO:**</font> Approximate the generalization performance of your best network. There is no code for this so far, you have to implement it.

In [ ]:
# Test your network's performance on the test data

### c) Visualizing kernels

<font color='red'>**TODO:**</font> Visualize the kernels of one of your models (do not use a model that has too many kernels, but at least 9). Write in your report what you observe looking at the kernels and try to speculate about the features they extract.

In [ ]:
def plot_kernels(network):
    kernels, bias = list(network.conv.parameters())
    kernels = kernels.detach().cpu()
    # scale between [-1,1]
    kernels = kernels / kernels.abs().reshape(len(kernels), -1).max(dim=1)[0].unsqueeze(1).unsqueeze(1).unsqueeze(1)

    plt.figure(figsize=(15,15))
    # plot kernels
    plt.imshow(
        torchvision.utils.make_grid(
            kernels,
            nrow = int(np.ceil(np.sqrt(len(kernels)))),
            pad_value = .0
        ).permute(1,2,0).mean(dim=2),
        cmap = "bwr",
        vmin = -1,
        vmax = 1
    )
    plt.axis("off")
    plt.colorbar()

In [ ]:
plot_kernels(network)

### d) Classify Learned Features with ML Models
<font color='red'>**TODO:**</font> Implement a function that - given a dataloader and network - extracts features (after the convolutional layer) for each image. Collect the features to generate training, validation, and test datasets. Use these datasets to train and evaluate your classifiers of Q1. Summarize your results in a table in your report and answer the following questions:
- How did the performance of each classifier evolve with the new set of features?
- Why do you think their performance improves / deteriorates (for each case)?

In [ ]:
def extract_features(dataloader, network):
    features = []
    targets = []
    
    for x, y in dataloader:
        targets += [y.detach().cpu().numpy()]
        ...
        
    return np.concatenate(features), np.concatenate(targets)

In [ ]:
# Generate datasets

In [ ]:
# Train models on generated datasets

### e) Deeper Learning
<font color='red'>**TODO:**</font> The network in Q2 is still quite shallow. You can increase its depth by, for example, adding more convolutional layers. Extend the code (or create a new class) such that you are able to add more convolutional and more linear layers. Train a deeper network. How good can your network become?

In [ ]:
# Conduct your experiments here